In [ ]:
from snowflake.snowpark.session import Session
session = get_active_session()

In [ ]:
from snowflake.snowpark.session import Session
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from wordcloud import WordCloud

sns.set_palette("viridis")

# Get Snowflake's active session
session = get_active_session()

# Load tables (using current DB + Schema from session context)
df_stocks = session.table("COMPANY_STOCKS")
df_reviews = session.table("COMPANY_REVIEWS")
df_stock_quotes = session.table("SILVER_CLEAN_STOCK_QUOTES")


In [ ]:
stocks = df_stocks.to_pandas()
reviews = df_reviews.to_pandas()
quotes = df_stock_quotes.to_pandas()

In [ ]:
stocks.head()

In [ ]:
reviews.head()

In [ ]:
quotes.head()

In [ ]:
# General overview of data
print("Stocks Data Overview:")
print(stocks.info())
print(stocks.describe(include='all'))

print("\nReviews Data Overview:")
print(reviews.info())
print(reviews.head())

print("\nStock Quotes Data Overview:")
print(quotes.info())
print(quotes.describe())

## Clean and Visualize Ratings and Review data

In [ ]:
# Split COMPANY_REVIEWS into Rating and Review text
reviews["RATING"] = reviews["COMPANY_REVIEWS"].str.extract(r'(\d+\.\d+)/5').astype(float)
reviews["REVIEW"] = reviews["COMPANY_REVIEWS"].str.replace(r'^\d+\.\d+/5\s*-\s*', '', regex=True)
reviews_cleaned = reviews[["SYMBOL","COMPANY_NAME","RATING","REVIEW"]]

In [ ]:
# Write the pandas DataFrame 'reviews_cleaned' to a Snowflake table
session.write_pandas(
    reviews_cleaned,
    table_name="COMPANY_REVIEWS_CLEANED",  # specify your target table name here
    auto_create_table=True,  # create table if it does not exist
    overwrite=True           # overwrite existing table data
)


In [ ]:
# Missing Values

print("\nMissing values in Stocks:\n", stocks.isnull().sum())
print("\nMissing values in Reviews:\n", reviews_cleaned.isnull().sum())

In [ ]:
# EDA on reviews
plt.figure(figsize=(8, 5))
sns.histplot(reviews_cleaned['RATING'].dropna(), bins=10, kde=True)
plt.title('Distribution of Company Reviews Ratings')
plt.xlabel('Rating')
plt.ylabel('Frequency')
plt.show()

## Wordcloud

In [ ]:
if "REVIEW" in reviews_cleaned.columns:
    text = " ".join(reviews_cleaned["REVIEW"].dropna().astype(str))
    wc = WordCloud(width=800, height=400, background_color="white").generate(text)

    plt.figure(figsize=(12,6))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title("Most Frequent Words in Reviews")
    plt.show()

In [ ]:
# Average rating by company
avg_rating = reviews_cleaned.groupby('SYMBOL')['RATING'].mean().reset_index()
plt.figure(figsize=(12, 6))
sns.barplot(x='SYMBOL', y='RATING', data=avg_rating)
plt.title('Average Company Rating by Symbol')
plt.xlabel('Company Symbol')
plt.ylabel('Average Rating')
plt.show()

### EDA on company stocks


In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(stocks['P_E_RATIO'].dropna(), bins=30, kde=True)
plt.title('Distribution of P/E Ratio')
plt.xlabel('P/E Ratio')
plt.ylabel('Frequency')
plt.show()

plt.figure(figsize=(10, 6))
sns.boxplot(x='SECTOR', y='P_E_RATIO', data=stocks)
plt.title('P/E Ratio by Sector')
plt.xticks(rotation=45)
plt.show()

## EDA on stock quotes


In [ ]:
plt.figure(figsize=(12, 6))
quotes['MARKET_TIMESTAMP'] = pd.to_datetime(quotes['MARKET_TIMESTAMP'])
quotes_2025 = quotes[quotes['MARKET_TIMESTAMP'].dt.year == 2025]
sns.boxplot(x='SYMBOL', y='CURRENT_PRICE', data=quotes_2025)
plt.xticks(rotation=45)
plt.title('2025 Stock Price Distribution by Company')
plt.xlabel('Company')
plt.ylabel('Current Price')
plt.show()


## Sentimental Analysis

In [ ]:
from textblob import TextBlob

# Define a function to get sentiment polarity using TextBlob
def get_sentiment_textblob(text):
    if isinstance(text, str):
        return TextBlob(text).sentiment.polarity
    else:
        return 0

# Apply function to reviews dataframe
reviews_cleaned = reviews_cleaned.copy()
reviews_cleaned['sentiment_score'] = reviews_cleaned['REVIEW'].apply(get_sentiment_textblob)

# Basic sentiment classification based on polarity score
reviews_cleaned['sentiment_label'] = reviews_cleaned['sentiment_score'].apply(lambda x:
                                                             'positive' if x > 0.05 else
                                                             ('negative' if x < -0.05 else 'neutral'))

# Average sentiment per company
avg_sentiment = reviews_cleaned.groupby('SYMBOL')['sentiment_score'].mean().reset_index()

# Plot average sentiment per company
plt.figure(figsize=(12, 6))
sns.barplot(x='SYMBOL', y='sentiment_score', data=avg_sentiment, color='steelblue')
plt.title('Average Sentiment Score per Company')
plt.xlabel('Company Symbol')
plt.ylabel('Average Sentiment Score')
plt.show()

# Plot sentiment label distribution
plt.figure(figsize=(10, 6))
sns.countplot(x='sentiment_label', data=reviews_cleaned, order=['positive', 'neutral', 'negative'], hue='sentiment_label', palette='viridis', dodge=False, legend=False)
plt.title('Sentiment Label Distribution in Reviews')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.show()
